# Stock Value Prediction

In this Notebook, we will create the actual prediction system, by testing various approaches and accuracy against multiple time-horizons (target_days variable).

First we will load all libraries:

In [2]:
import pandas as pd
import numpy as np
import sys, os
from datetime import datetime
sys.path.insert(1, '..')
import recommender as rcmd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# classification approaches
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

# regression approaches
from sklearn.linear_model import LinearRegression

# data handling and scoring
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, f1_score, mean_squared_error


Next, we create the input data pipelines for stock and statement data. Therefore we will have to split data into training and test sets. There are two options for doing that:

* Splitting the list of symbols
* Splitting the results list of training stock datapoints

We will use the first option in order ensure a clear split (since the generate data has overlapping time frames, the second options would generate data that might have been seen by the system beforehand).

In [3]:
# create cache object
cache = rcmd.stocks.Cache()

# load list of all available stocks and sample sub-list
stocks = cache.list_data('stock')

def train_test_data(back, ahead, xlim, split=0.3, count=2000, stocks=stocks, cache=cache):
    '''Generetes a train test split'''
    sample = np.random.choice(list(stocks.keys()), 2000)
    # split the stock data
    count_train = int((1-split) * count)
    sample_train = sample[:count_train]
    sample_test = sample[count_train:]

    # generate sample data
    df_train = rcmd.learning.preprocess.create_dataset(sample_train, stocks, cache, back, ahead, xlim)
    df_test = rcmd.learning.preprocess.create_dataset(sample_test, stocks, cache, back, ahead, xlim)
    
    return df_train, df_test

df_train, df_test = train_test_data(10, 22, (-.5, .5), split=0.2, count=4000)
print(df_train.shape)
df_train.head()

[-0.5  -0.25  0.    0.25  0.5 ]


..\recommender\learning\preprocess.py:259: RuntimeWarning: divide by zero encountered in true_divide
  df['expenses_research_netcash'] = np.divide(df['expenses_research'], df['cash_net'])
..\recommender\learning\preprocess.py:259: RuntimeWarning: invalid value encountered in true_divide
  df['expenses_research_netcash'] = np.divide(df['expenses_research'], df['cash_net'])
..\recommender\learning\preprocess.py:318: RuntimeWarning: divide by zero encountered in true_divide
  df['pe_ratio'] = np.divide(df[col_price], df['eps_diluted'])
..\recommender\learning\preprocess.py:319: RuntimeWarning: divide by zero encountered in true_divide
  df['cash_share'] = np.divide(df['cash_net'], np.divide(df['shareholder_equity'], df[col_price]))
..\recommender\stocks\Statements.py:66: UserWarning: No statements found for given symbols
  warnings.warn("No statements found for given symbols")


ValueError: No objects to concatenate

In [3]:
# shortcut: store / load created datasets
df_train.to_csv('../data/train.csv')
df_test.to_csv('../data/test.csv')

# load data
#df_train = pd.read_csv('../data/train.csv')
#df_test = pd.read_csv('../data/test.csv')

Now that we have loaded and split the data, we have to divide it into input and output data:

In [ ]:
def divide_data(df, xlim, balance_mode=None, balance_weight=1):
    '''Splits the data into 3 sets: input, ouput_classify, output_regression.
    
    Note that this function will also sample the data if choosen to create a more balanced dataset. Options are:
        `under`: Undersamples the data (takes lowest data sample and )
        `over`: Oversamples data to the highest number of possible samples
        `over_under`: takes the mean count and samples in both directions
        
    Args:
        df (DataFrame): DF to contain all relevant data
        xlim (tuple): tuple of integers used to clip and scale regression values to a range of 0 to 1
        balance_mode (str): Defines the balance mode of the data (options: 'over_under', 'under', 'over')
        balance_weight (float): Defines how much the calculate sample count is weighted in comparision to the actual count (should be between 0 and 1)
        
    Returns:
        df_X: DataFrame with input values
        df_y_cls: DataFrame with classification labels
        df_y_reg: DataFrame with regression values
    '''
    # sample the data correctly
    if balance_mode is not None:
        if balance_mode == 'over_under':
            # find median
            num_samples = df['target_cat'].value_counts().median().astype('int')
        elif balance_mode == 'over':
            # find highest number
            num_samples = df['target_cat'].value_counts().max()
        elif balance_mode == 'under':
            # find minimal number
            num_samples = df['target_cat'].value_counts().min()
        else:
            raise ValueError('Unkown sample mode: {}'.format(balance_mode))
        # sample categories
        dfs = []
        for cat in df['target_cat'].unique():
            df_tmp = df[df['target_cat'] == cat]
            cur_samples = int(balance_weight * num_samples + (1-balance_weight) * df_tmp.shape[0])
            sample = df_tmp.sample(cur_samples, replace=cur_samples > df_tmp.shape[0])
            dfs.append(sample)
        # concat and shuffle the rows
        df = pd.concat(dfs, axis=0).sample(frac=1)
    
    # remove all target cols
    df_X = df.drop(['target', 'target_cat', 'norm_price', 'symbol'], axis=1)
    # convert to dummy classes
    df_y_cls = pd.get_dummies(df['target_cat'], prefix='cat', dummy_na=False)
    # clip values and scale to vals
    df_y_reg = np.divide( np.subtract( df['target'].clip(xlim[0], xlim[1]), xlim[0] ), (xlim[1] - xlim[0]) )
    
    return df, df_X, df_y_cls, df_y_reg

df_train_bm, X_train, y_ctrain, y_rtrain = divide_data(df_train, (-.5, .5), balance_mode='over_under', balance_weight=0.9)
df_test_bm, X_test, y_ctest, y_rtest = divide_data(df_test, (-.5, .5))
print(pd.concat([y_ctrain.sum(axis=0), y_ctest.sum(axis=0)], axis=1))

Before we create the actual prediction systems, we will have to define metrics, how we want to measure the success of the systems.
As we have two approaches (classification and regression) we will use two types metrics:

* Precision, Recall & Accuracy
* RMSE

In [ ]:
def _metric_classifier(y_true, y_pred, avg=None):
    p = precision_score(y_true, y_pred, average=avg)
    r = recall_score(y_true, y_pred, average=avg)
    f1 = f1_score(y_true, y_pred, average=avg)
    return f1, p, r

def score_classifier(y_true, y_pred):
    '''Calculates the relevant scores for a classifer and outputs. This should show predicitions per class.'''
    f1, p, r = _metric_classifier(y_true, y_pred, avg='micro')
    
    print("Model Performance: F1={:.4f} (P={:.4f} / R={:.4f})".format(f1, p, r))
    # list scores of single classes
    for i, c in enumerate(y_true.columns):
        sf1, sp, sr = _metric_classifier(y_true.iloc[:, i], y_pred[:, i], avg='binary')
        print("  {:10} F1={:.4f} (P={:.4f} / R={:.4f})".format(c + ":", sf1, sp, sr))
        
def score_regression(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print("Model Performance: MSE={:.4f}".format(mse))

## Classification

The first step is to create a baseline for both approaches (classification and regression). In case of regression our target value will be `target` and for classification it will be `target_cat` (which we might convert into a one-hot vector along the way).

Lets start with the simpler form of classification:

In [9]:
y_ctrain.sum(axis=0)

cat_0    2818
cat_1    2818
cat_2    2818
cat_3    2818
cat_4    2818
cat_5    2818
dtype: int64

In [ ]:
# scale input data to improve convergance (Note: scaler has to be used for other input data as well)
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# train element
classifier = MultiOutputClassifier(LogisticRegression(max_iter=500, solver='lbfgs'))
classifier.fit(X_train_std, y_ctrain)

# predict data
y_pred = classifier.predict(X_test_std)

score_classifier(y_ctest, y_pred)

We can see a strong bias in the system for `cat_3`, which also has the highest number of training samples. Future work might include oversampling or more target datapoint selection to reduce these biases.

Next, support vector machines:

In [20]:
classifier_svm = MultiOutputClassifier(SVC())
classifier_svm.fit(X_train_std, y_ctrain)
y_pred_svm = classifier_svm.predict(X_test_std)
score_classifier(y_ctest, y_pred_svm)

Model Performance: F1=0.4157 (P=0.5416 / R=0.3372)
  cat_0:     F1=0.0385 (P=1.0000 / R=0.0196)
  cat_1:     F1=0.0154 (P=1.0000 / R=0.0078)
  cat_2:     F1=0.0107 (P=0.4932 / R=0.0054)
  cat_3:     F1=0.6126 (P=0.5414 / R=0.7053)
  cat_4:     F1=0.0027 (P=1.0000 / R=0.0014)
  cat_5:     F1=0.0000 (P=0.0000 / R=0.0000)


c:\programming\envs\ds-stocks\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\ds-stocks\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


We can see the results improve

In [ ]:
class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, data=X_test_std):
        self.data = data
    def on_epoch_end(self, epoch, logs={}):
        loss, acc = self.model.evaluate(self.data, df_test_bm['target_cat'].to_numpy(), verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
# simple feed forward network
print(X_train.shape)
print(df_train.shape)
classifier_ffn = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train_std.shape[1],)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(y_ctrain.shape[1], activation=tf.nn.softmax)
])
classifier_ffn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier_ffn.fit(X_train.to_numpy(), df_train_bm['target_cat'].to_numpy(), epochs=100, callbacks=[TestCallback()])

y_pred_ffn = classifier_ffn.predict(X_test.to_numpy())
y_pred_ffn = pd.get_dummies(y_pred_ffn.argmax(axis=1))
print(y_pred_ffn.sum(axis=0))
score_classifier(y_ctest, y_pred_ffn.to_numpy())

It is noteworthy that the output of the model in the test data resembles the input distribution. Lets try to improve generalization with a more complex model

In [ ]:
act = tf.keras.layers.PReLU
classifier_ffn = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train_std.shape[1],)),
    tf.keras.layers.Dense(32), act(),
    tf.keras.layers.Dense(64), act(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128), act(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256), act(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128), act(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64), act(),
    tf.keras.layers.Dense(y_ctrain.shape[1], activation=tf.nn.softmax)
])
classifier_ffn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier_ffn.fit(X_train.to_numpy(), df_train_bm['target_cat'].to_numpy(), epochs=200, callbacks=[TestCallback(X_test.to_numpy())])

y_pred_ffn = classifier_ffn.predict(X_test.to_numpy())
print(y_pred_ffn)
y_pred_ffn = pd.get_dummies(y_pred_ffn.argmax(axis=1))
print(y_pred_ffn.sum(axis=0))
score_classifier(y_ctest, y_pred_ffn.to_numpy())

In [32]:
# save the model
classifier_ffn.save('../data/keras-model.h5')

## Regression

The other possible option is regression. We will test a linear regression against neural networks based on RMSE score to see how the predictions hold.

In [39]:
reg = LinearRegression()
reg.fit(X_train.iloc[:, :7].to_numpy(), y_rtrain)
y_pred_reg = reg.predict(X_test.iloc[:, :7].to_numpy())
score_regression(y_rtest, y_pred_reg)

Model Performance: MSE=0.0329


Now the neural Network

In [50]:
classifier_reg = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train_std.shape[1],)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])
opt = tf.keras.optimizers.SGD(learning_rate=0.00000001, nesterov=False)
classifier_reg.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])
classifier_reg.fit(X_train.to_numpy(), y_rtrain.to_numpy(), epochs=20)

y_pred_reg = classifier_reg.predict(X_test.to_numpy())
score_regression(y_rtest, y_pred_reg)

Train on 42542 samples
Epoch 1/20
42542/42542 [==============================] - 8s 181us/sample - loss: 72308.8231 - accuracy: 0.0097
Epoch 2/20
42542/42542 [==============================] - 6s 146us/sample - loss: 3158.4172 - accuracy: 0.0091
Epoch 3/20
42542/42542 [==============================] - 7s 155us/sample - loss: 7.5809 - accuracy: 0.0061
Epoch 4/20
42542/42542 [==============================] - 7s 156us/sample - loss: 5.0027 - accuracy: 0.0062
Epoch 5/20
42542/42542 [==============================] - 6s 151us/sample - loss: 4.0884 - accuracy: 0.0061
Epoch 6/20
42542/42542 [==============================] - 7s 161us/sample - loss: 3.5077 - accuracy: 0.0063
Epoch 7/20
42542/42542 [==============================] - 6s 140us/sample - loss: 2.9895 - accuracy: 0.0062
Epoch 8/20
42542/42542 [==============================] - 7s 165us/sample - loss: 2.7040 - accuracy: 0.0061
Epoch 9/20
42542/42542 [==============================] - 7s 169us/sample - loss: 2.4751 - accuracy: 0.006

Model Performance: MSE=3.2631


In [51]:
y_pred_reg

array([[ 0.10986691],
       [ 0.23991095],
       [-0.34694692],
       ...,
       [ 0.0956597 ],
       [-0.04288995],
       [ 0.15074253]], dtype=float32)

In [52]:
y_pred_reg.shape

(18622, 1)

In [53]:
y_pred_ffn = classifier_ffn.predict(X_test.to_numpy())
print(y_pred_ffn)

[[0.00190905 0.0575433  0.4032011  0.48665118 0.04683916 0.00385619]
 [0.00347802 0.07319234 0.39260495 0.47064242 0.05408182 0.00600046]
 [0.00484556 0.07138    0.38683167 0.45450288 0.06774367 0.01469623]
 ...
 [0.00111771 0.03497788 0.40016484 0.5337459  0.0289243  0.00106938]
 [0.00483004 0.07679388 0.38836578 0.45680222 0.06269442 0.01051365]
 [0.00092138 0.0323066  0.41052336 0.5202334  0.03445359 0.00156169]]
